# Location Extraction for NASA Social Landslides
- Building a logistic regression model to detect whether a sentence contains the gold location description keywords
- Train on the processed NASA example `new_filter_train_set.csv` and predicted on the `article_sample.tsv`.

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
import geocoder
import geopy.distance
import pickle
import math
from datetime import datetime

In [2]:
import sklearn
print('The sklearn version is {}.'.format(sklearn.__version__))

The sklearn version is 1.1.1.


## 1. Data Cleaning

- Get the POS/NEG label for each sentence
    - First do exact match: if `location_description` appears in `text`, mark the sentence as a positive sentence (`pos_setence=Yes`), otherwise, mark the sentence as a negative sentence (`pos_setence=No`).
    - Then do partial match: if one or more location entities in the sentence appear in the gold label (`location_description`), mark the sentence as a positive sentence (`pos_setence=Yes`), otherwise, mark the sentence as a negative sentence (`pos_setence=No`).

In [3]:
ner = pd.read_csv('../data/new_filter_train_set.csv', index_col=0) 
ner

,id,text,GPE,LOC,DATE,TIME
0,0,Reported By: | Edited By: |Source: ANI |Update...,NaN,NaN,"Aug 04 , 2015",11:39 AM IST
1,0,At least 12 people are feared to be buried und...,Dabhol|Maharashtra|Ratnagiri,NaN,NaN,NaN
2,0,The landslide is said to have taken place at 3...,NaN,NaN,NaN,3:30 am
3,0,Ratnagiri (Maharashtra): Landslide occurred at...,Ratnagiri|Maharashtra,NaN,NaN,3 : 30 am today
4,0,"pic.twitter.com/MOcmTxnAZX June 22, 2015",NaN,NaN,"June 22 , 2015",NaN
...,...,...,...,...,...,...
26128,35646,"Meanwhile, the Kanchanjunga Academy Higher Sec...",Phidim-1,NaN,NaN,NaN
26129,35649,Follow us on landslide blocks doda batote high...,Jammu,NaN,NaN,NaN
26130,35649,The highway was blocked after a landslide hit ...,Doda,NaN,NaN,last evening
26131,35649,He said that the work to open the highway was ...,NaN,NaN,today,evening


In [4]:
def merge_locs_dates(data):
    """Fill NAs and merge location/date columns"""
    data = data.fillna('')  # replace NAN with empty string
    data['locations'] = data[['GPE', 'LOC']].agg('|'.join, axis=1)  # join GPE and LOC by |
    data['dates'] = data[['DATE','TIME']].agg('|'.join, axis=1)  # join DATE and TIME by |
    data = data.drop(columns=['GPE', 'LOC', 'DATE', 'TIME'])  # keep only the joined column
    return data

In [5]:
def preprocess(nasa, ner):
    """
    prepare data for pos sentence prediction model
    
    Parameters
    ----------
    nasa : pandas DataFrame
        nasa dataset containing location_description
    ner : pandas DataFrame
        train dataset containing id, text, GPE, LOC, DATE, TIME columns

    Returns
    ----------
        a pandas DataFrame with id, text, location_description, locations, dates, pos_sentence columns
    """
    nasa = nasa.reset_index()
    nasa = nasa.rename(columns={"index": "id"})  # id: index in NASA dataset
    df = pd.merge(ner, nasa, how='left', on='id')
    df = merge_locs_dates(df[['id', 'text', 'location_description', 'GPE', 'LOC', 'DATE', 'TIME']])
    
    data = df.to_dict()  # transform dataframe into dictionary

    data['pos_sentence'] = {}
    n = defaultdict(int)
    data['number_of_pos_sent'] = {}
    data['contain_pos_sent'] = {}

    # exact match
    # iterate over each sentence in the data
    for i in range(len(data['text'])):
        if data['location_description'][i] in data['text'][i]:
            data['pos_sentence'][i] = 'Yes'
            n[data['id'][i]] += 1
        else:
            data['pos_sentence'][i] = 'No'

    # partial match
    for i in range(len(data['text'])):
        if n[data['id'][i]] == 0:
            locs = list(filter(None, data['locations'][i].split("|"))) 
            if any(loc in data['location_description'][i] for loc in locs):
                data['pos_sentence'][i] = 'Yes'
                n[data['id'][i]] += 1
            else:
                data['pos_sentence'][i] = 'No'
    
    # count how many pos_sentence each document has
    # count how many documents contain gold place name, how many doesn't
    for i in range(len(data['text'])):
        if n[data['id'][i]] == 0:
            data['number_of_pos_sent'][i] = 0
            data['contain_pos_sent'][i] = False
        else:
            data['number_of_pos_sent'][i] = n[data['id'][i]]
            data['contain_pos_sent'][i] = True
        
    return pd.DataFrame(data)[pd.DataFrame(data)['contain_pos_sent']]

In [6]:
nasa = pd.read_csv("../data/nasa_global_landslide_catalog_point.csv")
ner = pd.read_csv('../data/new_filter_train_set.csv', index_col=0) 
data = preprocess(nasa, ner)
data

,id,text,location_description,locations,dates,pos_sentence,number_of_pos_sent,contain_pos_sent
0,0,Reported By: | Edited By: |Source: ANI |Update...,Dabhol village of Maharashtra's Ratnagiri dist...,|,"Aug 04 , 2015|11:39 AM IST",No,1,True
1,0,At least 12 people are feared to be buried und...,Dabhol village of Maharashtra's Ratnagiri dist...,Dabhol|Maharashtra|Ratnagiri|,|,Yes,1,True
2,0,The landslide is said to have taken place at 3...,Dabhol village of Maharashtra's Ratnagiri dist...,|,|3:30 am,No,1,True
3,0,Ratnagiri (Maharashtra): Landslide occurred at...,Dabhol village of Maharashtra's Ratnagiri dist...,Ratnagiri|Maharashtra|,|3 : 30 am today,No,1,True
4,0,"pic.twitter.com/MOcmTxnAZX June 22, 2015",Dabhol village of Maharashtra's Ratnagiri dist...,|,"June 22 , 2015|",No,1,True
...,...,...,...,...,...,...,...,...
26128,35646,"Meanwhile, the Kanchanjunga Academy Higher Sec...",Phidim-Raanke section of the Mechi Highway at ...,Phidim-1|,|,No,1,True
26129,35649,Follow us on landslide blocks doda batote high...,"Chakwa bridge, 9 kilomters short of Doda",Jammu|,|,No,1,True
26130,35649,The highway was blocked after a landslide hit ...,"Chakwa bridge, 9 kilomters short of Doda",Doda|,|last evening,Yes,1,True
26131,35649,He said that the work to open the highway was ...,"Chakwa bridge, 9 kilomters short of Doda",|,today|evening,No,1,True


In [7]:
describe_df = data[['id', 'number_of_pos_sent', 'contain_pos_sent']].drop_duplicates()
print(f"Distribution of number of positive sentence each document has: \n{describe_df['number_of_pos_sent'].value_counts().sort_index()}\n")
print(f"How many documents contain gold place name: \n{describe_df['contain_pos_sent'].value_counts()}")

Distribution of number of positive sentence each document has: 
1     2498
2       74
3       29
4       11
5        6
6        6
7        2
8        1
9        4
10       3
11       1
12       1
13       4
18       1
19       1
Name: number_of_pos_sent, dtype: int64

How many documents contain gold place name: 
True    2642
Name: contain_pos_sent, dtype: int64


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22354 entries, 0 to 26132
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    22354 non-null  int64 
 1   text                  22354 non-null  object
 2   location_description  22354 non-null  object
 3   locations             22354 non-null  object
 4   dates                 22354 non-null  object
 5   pos_sentence          22354 non-null  object
 6   number_of_pos_sent    22354 non-null  int64 
 7   contain_pos_sent      22354 non-null  bool  
dtypes: bool(1), int64(2), object(5)
memory usage: 1.4+ MB


In [9]:
data.shape

(22354, 8)

## 2. Train the Model

In [10]:
def get_table(ids, table):
    """split the data into train, dev, test according to ids files"""
    ids = pd.read_csv(f'../data/{ids}')
    ids = ids.iloc[:,1:]
    ids = ids.rename(columns={'ids':'id'})
    ids['yes'] = 1
    data = pd.merge(table, ids, how="left", on="id")
    return data.query('yes == yes').iloc[:,:-1]

train = get_table("train_ids.csv", data)
dev = get_table("dev_ids.csv", data)
test = get_table("test_ids.csv", data)

In [11]:
def loc_train(data):
    train = get_table("train_ids.csv", data)
    dev = get_table("dev_ids.csv", data)
    test = get_table("test_ids.csv", data)
    X_train, y_train = train["text"], train["pos_sentence"]
    X_dev, y_dev = dev["text"], dev["pos_sentence"]
    X_test, y_test = test["text"], test["pos_sentence"]

    model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), LogisticRegression(max_iter=2000, class_weight='balanced'))
    model.fit(X_train, y_train)
    
    dev_scores = classification_report(y_dev, model.predict(X_dev))
    test_scores = classification_report(y_test, model.predict(X_test))

    return model, dev_scores, test_scores

model, dev_s, test_s = loc_train(data)
print(f"dev: \n {dev_s}")
print(f"test: \n {test_s}")

dev: 
               precision    recall  f1-score   support

          No       0.92      0.78      0.84      1931
         Yes       0.31      0.61      0.41       322

    accuracy                           0.75      2253
   macro avg       0.62      0.69      0.63      2253
weighted avg       0.84      0.75      0.78      2253

test: 
               precision    recall  f1-score   support

          No       0.90      0.81      0.85      1739
         Yes       0.33      0.50      0.40       329

    accuracy                           0.76      2068
   macro avg       0.61      0.65      0.62      2068
weighted avg       0.81      0.76      0.78      2068



In [12]:
pickle.dump(model, open('../data/results/loc_model', 'wb'))  # store the model

## 3. Prediction

In [13]:
def get_distance(p1, p2):
    """Get the geographical distance between two points"""
    if p1 and p2:
        return round(geopy.distance.geodesic(p1, p2).km, 3)
    else:
        return None

def get_radius(p1, p2):
    """Get the radius of a region"""
    if p1 and p2:
        return round(geopy.distance.geodesic(p1, p2).km, 3)/2
    else:
        return None

In [14]:
def get_outlier_idx(centroid, points):
    """
    Parameters: 
        centroid: a tuple of centroid;
        points: a list of tuples
    Return:
        the index of the point that should be removed
    """
    dists = [get_distance(centroid, point) for point in points]
    return dists.index(max(dists))

In [15]:
def get_smallest_region_idx(locs):
    """
    Parameters
    ----------
    locs : list of dictionary
        a list of dictionary containing latitude, longitude, 
        northeast point, southwest point for all the location 
        entities in the positive sentence
    
    Returns
    ----------
        an integer indicating the index of the location entity 
        that has the smallest region
    """
    dists = [get_distance(loc['northeast'], loc['southwest']) for loc in locs]
    return dists.index(min(dists))

In [16]:
def loc_predict(df, model): 
    """Get the most likely locations, latitude, longitude based on pred model

    Parameters
    ----------
    df: 
        a data frame containing document ID (id) and tokenized sentences (text) for each document, 
        extracted location entities (locations), and extracted date entities (dates)
    model:
        the prediction model (logistic model trained on NASA dataset)

    Returns
    -------
        a data frame with locations, the most likely location, latitude, longitude, diameter
    """
#     df = merge_locs_dates(df)
    
    # get predict_proba
    pd.options.mode.chained_assignment = None   # silent warning message
    df['predict_proba'] = model.predict_proba(df['text'])[:, 1]

    result = {'locations': defaultdict(str),
              'location': defaultdict(str),
              'latitude': defaultdict(float),
              'longitude': defaultdict(float),
              'radius_km': defaultdict(float)}

    # get a dict of idxmax for each document
    idx_max = df.groupby('id')['predict_proba'].idxmax().to_dict()

    data = df.to_dict()
    for i, idx in idx_max.items():  # i: index of the document; idx: index of the df
        # ensure the `locations` column of the `idxmax` row is not empty 
        current_proba = df.query('id == @i')['predict_proba']
        while data['locations'][idx] == "|":
            try:
                current_proba = current_proba.drop(idx)  # drop the current idxmax
                idx = current_proba.idxmax()  # get the idxmax of the rest
            except ValueError:
                print(f"All locations in document {i} are empty!") 
                idx = -1  # set idx=-1 if all locations are empty
                break
        
        # store the locations, latitude, longitude in result dict
        if idx != -1:
            result['locations'][i] = data['locations'][idx]

            locs = list(filter(None, data['locations'][idx].split("|")))
            geolocs = []
            for loc in locs:
                geocoded = geocoder.arcgis(loc).json
                if geocoded:
                    geoloc = geocoded['bbox']
                    geoloc['lat'], geoloc['lng'] = geocoded['lat'], geocoded['lng']
                    geolocs.append(geoloc)
            
            if len(geolocs) > 2:
                # remove the farthest outlier
                lats, lngs = [loc['lat'] for loc in geolocs], [loc['lng'] for loc in geolocs]
                mean_lat, mean_lng = np.mean(lats), np.mean(lngs)  # get the centroid
                x = get_outlier_idx(
                    (mean_lat, mean_lng), 
                    [(lat, lng) for lat, lng in zip(lats, lngs)]
                )
                del geolocs[x]
                del locs[x]
                # get the index of location with the smallest region
                j = get_smallest_region_idx(geolocs)
                location = locs[j]
                lat, lng = geolocs[j]['lat'], geolocs[j]['lng']
                ne, sw = geolocs[j]['northeast'], geolocs[j]['southwest']
            elif len(geolocs) == 2:
                j = get_smallest_region_idx(geolocs)
                location = locs[j]
                lat, lng = geolocs[j]['lat'], geolocs[j]['lng']
                ne, sw = geolocs[j]['northeast'], geolocs[j]['southwest']
            elif len(geolocs) == 1:
                location = locs[0]
                lat, lng = geolocs[0]['lat'], geolocs[0]['lng']
                ne, sw = geolocs[0]['northeast'], geolocs[0]['southwest']
            else:
                print(f"Locations in document {i} cannot be geocoded!")
                location, lat, lng, ne, sw = None, None, None, None, None
        else:
            result['locations'][i] = None
            location, lat, lng, ne, sw = None, None, None, None, None

        result['location'][i] = location
        result['latitude'][i], result['longitude'][i] = lat, lng
        result['radius_km'][i] = get_radius(ne, sw)

    return pd.DataFrame(result)

In [17]:
dev_result = loc_predict(dev, model) 
dev_result

Locations in document 3002 cannot be geocoded!


,locations,location,latitude,longitude,radius_km
124,GUNDOGDU|Turkey|Turkey|,Turkey,39.066251,35.142286,893.1935
164,St. Thomas|Mahagony Vale,Mahagony Vale,3.132850,101.671090,0.7840
166,|North Asheville,North Asheville,35.598680,-82.553400,15.9015
288,Bybrook|Portland|,Bybrook,51.161387,0.880618,0.7885
321,Yasamal|,Yasamal,40.381530,49.807470,1.3980
...,...,...,...,...,...
34962,New Delhi|Beyul Pemako|the India-China border,Beyul Pemako,16.695800,74.241810,0.7685
34980,Kamloops|Merritt|Campbell Creek|Separation|,Separation,-26.078130,27.968940,0.7465
35592,Darkha VDC-8|Dhading|,Darkha VDC-8,24.944620,85.932070,1.4990
35633,Surigao del Sur|DAVAO CITY|Philippines|Surigao...,DAVAO CITY,7.065740,125.610800,46.7400


In [18]:
test_result = loc_predict(test, model) 
test_result

Locations in document 5397 cannot be geocoded!


,locations,location,latitude,longitude,radius_km
52,Pokhara-Baglung-Jomsom|Pokhara-Baglung-Jomsom|,Pokhara-Baglung-Jomsom,28.264860,83.411206,0.1480
90,Alpine|,Alpine,32.834660,-116.752390,1.4515
346,California-esque|,California-esque,36.374106,-119.270230,703.6710
348,|North Rodeo Gulch,North Rodeo Gulch,37.002821,-121.969598,0.1425
363,Kishtwar|Doda-Kishtwar|,Kishtwar,33.316240,75.765130,32.1495
...,...,...,...,...,...
34734,Guatemala|,Guatemala,15.697238,-90.363979,311.3200
34858,Mexico|Oaxaca|,Oaxaca,17.061730,-96.726100,9.3670
34901,Kishtwar|Kuligardh Drabshallah,Kishtwar,33.316240,75.765130,32.1495
34977,|Big Sur,Big Sur,36.270680,-121.808150,1.4280


In [19]:
dev_result.to_csv('../data/results/logistic_dev_result.csv')
test_result.to_csv('../data/results/logistic_test_result.csv')

## 4. Evaluate

In [20]:
def get_distance2(pred_lat, pred_lng, gold_lat, gold_lng):
    if pd.isnull(pred_lat):
        return None
    else:
        return round(geopy.distance.geodesic((pred_lat, pred_lng), (gold_lat, gold_lng)).km, 3)
    
def get_gold_radius(accuracy):
    if not pd.isnull(accuracy):
        if accuracy == 'exact' or accuracy == 'Known exactly':
            output = 0.1
        if accuracy.lower() == 'unknown':
            output = 100
        if accuracy.endswith('km'):
            output = float(accuracy[:-2])
    else:
        output = None
    return output

def get_correct(r_pred, r_gold, d):
    if d <= r_pred + r_gold:
        correct = True
    else:
        correct = False
    return correct

def get_precision_recall_f1(r_pred, r_gold, d):
    """Get location evaluation metrics based on intersected area"""
    if r_pred >= r_gold:
        r1 = r_pred
        r2 = r_gold
    else:
        r1 = r_gold
        r2 = r_pred

    if d >= r1 + r2:
        a_intersection = 0
    elif d <= r1 - r2:
        a_intersection = math.pi * (r2**2)
    else:
        d1 = (r1**2 - r2**2 + d**2)/(2*d)
        d2 = d - d1
        a1 = (r1**2) * math.acos(d1/r1) - d1 * math.sqrt(r1**2 - d1**2)
        a2 = (r2**2) * math.acos(d2/r2) - d2 * math.sqrt(r2**2 - d2**2)
        a_intersection = a1 + a2

    a_pred = math.pi * (r_pred**2)
    a_gold = math.pi * (r_gold**2)

    precision = a_intersection/a_pred
    recall = a_intersection/a_gold
    
    if precision != 0 and recall != 0:
        f1 = 2*precision*recall/(precision+recall)
    else:
        f1 = 0
    
    return precision, recall, f1

In [21]:
def get_evaluate_table(pred, gold):  # pred: pred_df, gold: nasa dataset
    pred = pred.set_index('id')
    gold = gold.rename(columns={"latitude": "gold_latitude", "longitude": "gold_longitude"})
    data = pred.join(gold[['location_description', 'location_accuracy', 'gold_latitude', 'gold_longitude']])
    data = data.assign(distance_km=data.apply(lambda x: get_distance2(x.latitude, x.longitude, x.gold_latitude, x.gold_longitude), axis=1))  
    data = data.assign(gold_radius_km=data.apply(lambda x: get_gold_radius(x.location_accuracy), axis=1))
    data = data.assign(correct=data.apply(lambda x: get_correct(x.radius_km, x.gold_radius_km, x.distance_km), axis=1))
    data[['precision', 'recall', 'f1_score']] = data.apply(lambda x: get_precision_recall_f1(x.radius_km, x.gold_radius_km, x.distance_km), axis=1, result_type='expand')
    return data

In [22]:
def get_evaluation(path, gold):
    data = pd.read_csv(path)
    data = data.rename(columns={'Unnamed: 0':'id'})
    result = get_evaluate_table(data, gold)
    print(f"Model prediction accuracy: {result['correct'].sum()/len(result)}")
    return result.query('precision != 0').describe()[['precision', 'recall', 'f1_score']]

In [23]:
gold = pd.read_csv("../data/nasa_global_landslide_catalog_point.csv")

In [24]:
get_evaluation("../data/results/logistic_dev_result.csv", gold)  # Development/Validation accuracy

Model prediction accuracy: 0.4708029197080292


,precision,recall,f1_score
count,1.290000e+02,129.000000,1.290000e+02
mean,3.264869e-01,0.706763,1.383694e-01
std,4.160455e-01,0.409211,1.993524e-01
min,5.276656e-08,0.002190,1.055331e-07
25%,2.151607e-03,0.201529,4.293975e-03
50%,5.801883e-02,1.000000,4.326668e-02
75%,7.498490e-01,1.000000,1.799482e-01
max,1.000000e+00,1.000000,8.664580e-01


In [25]:
get_evaluation("../data/results/logistic_test_result.csv", gold)  # Test accuracy

Model prediction accuracy: 0.44528301886792454


,precision,recall,f1_score
count,1.180000e+02,118.000000,1.180000e+02
mean,3.036881e-01,0.734872,1.636953e-01
std,4.013659e-01,0.383069,2.224368e-01
min,9.378043e-09,0.000199,1.875609e-08
25%,1.288314e-03,0.441029,2.158565e-03
50%,5.417733e-02,1.000000,4.669371e-02
75%,6.165516e-01,1.000000,2.599918e-01
max,1.000000e+00,1.000000,8.844643e-01
